In [1]:
# JUST RUN ALL CELLS AND CHECK THE RESULTS AT THE BOTTOM

import argparse
import csv
import os
import shutil
from tqdm import tqdm

try: import setGPU
except ImportError: pass

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import numpy as np
import numpy.random as npr

import sys
import time

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('bmh')

import models

import sys

In [2]:
model = torch.load("work/2.optnetIneq.Qpenalty=0.1.nineq=100/latest.pth", map_location=torch.device('cpu'))

In [3]:
with open('data/{}/features.pt'.format(2), 'rb') as f:
    testX = torch.load(f)
with open('data/{}/labels.pt'.format(2), 'rb') as f:
    testY = torch.load(f)

In [4]:
def computeErr(pred):
    batchSz = pred.size(0)
    nsq = int(pred.size(1))
    n = int(np.sqrt(nsq))
    s = (nsq-1)*nsq//2 # 0 + 1 + ... + n^2-1
    I = torch.max(pred, 3)[1].squeeze().view(batchSz, nsq, nsq)

    def invalidGroups(x):
        valid = (x.min(1)[0] == 0)
        valid *= (x.max(1)[0] == nsq-1)
        valid *= (x.sum(1) == s)
        return ~valid

    boardCorrect = torch.ones(batchSz).type_as(pred)
    for j in range(nsq):
        # Check the jth row and column.
        boardCorrect[invalidGroups(I[:,j,:])] = 0
        boardCorrect[invalidGroups(I[:,:,j])] = 0

        # Check the jth block.
        row, col = n*(j // n), n*(j % n)
        M = invalidGroups(I[:,row:row+n,col:col+n].contiguous().view(batchSz,-1))
        boardCorrect[M] = 0

        if boardCorrect.sum() == 0:
            return batchSz

    return batchSz-boardCorrect.sum().item()

In [5]:
def test(model, batchSz = 100):
    test_loss = 0
    batch_data_t = torch.FloatTensor(batchSz, testX.size(1), testX.size(2), testX.size(3))
    batch_targets_t = torch.FloatTensor(batchSz, testY.size(1), testX.size(2), testX.size(3))

    batch_data = Variable(batch_data_t)
    batch_targets = Variable(batch_targets_t)

    nErr = 0
    start_time = time.time()
    size = testX.size(0)
    
    with torch.no_grad():
        for i in range(0, size, batchSz):
            print('Testing model: {}/{}'.format(i, size), end='\r')
            batch_data.data[:] = testX[i:i+batchSz]
            batch_targets.data[:] = testY[i:i+batchSz]
            output = model(batch_data)
            test_loss += nn.MSELoss()(output, batch_targets)
            nErr += computeErr(output.data)

    dt = time.time() - start_time
    
    nBatches = size/batchSz
    test_loss = test_loss.item()/nBatches
    test_err = nErr/size
    print('TEST SET RESULTS:' + ' ' * 20)
    print('Average loss: {:.4f}'.format(test_loss))
    print('Err: {:.4f}'.format(test_err))
    print(f'nErr: {nErr}')
    
    print(f'Average time per batch: {dt / nBatches}')


In [6]:
model1 = models.DynamicIneq(n=2, Qpenalty=0.1, nineq=100)

params = model.named_parameters()
params1 = model1.named_parameters()

dict_params1 = dict(params1)

for name, param in params:
    if name in dict_params1:
        dict_params1[name].data.copy_(param.data)

In [7]:
# THIS TESTS OPTNET
test(model)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys


TEST SET RESULTS:                    
Average loss: 0.0055
Err: 0.0030
nErr: 3.0
Average time per batch: 13.113858509063721


In [8]:
# THIS TESTS DYNAMIC
test(model1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys


TEST SET RESULTS:                    
Average loss: inf
Err: 1.0000
nErr: 1000
Average time per batch: 0.0841663122177124
